# Front-End Communication (or Supporting Widgets)

See detailed documentation in `docs/FrontEndCommunication.md`.

In [1]:
%goflags --cover --covermode=set

%goflags=["--cover" "--covermode=set"]


In [2]:
!if [[ "${GONB_GIT_ROOT}" == "" ]] ; then \
    echo "Please set GONB_GIT_ROOT before runnig this notebook!" 1>&2 ; \
else \
    echo "ok" ; \
fi

ok


In [3]:
!*rm -f go.work && go work init && go work use . "${GONB_GIT_ROOT}"
%goworkfix

	- Added replace rule for module "github.com/janpfeifer/gonb" to local directory "/home/janpf/Projects/gonb".


### Test Initialization of WebSocket

In [4]:
%widgets

### Send incrementing counter back-and-forth to front-end

In [5]:
import (
    "github.com/janpfeifer/gonb/gonbui/comms"
    "github.com/janpfeifer/gonb/gonbui/dom"

    "log"
    "os"
    "time"
)

const (
    toFrontEnd = "/nbtest/to_frontend"
    toCell = "nbtest/to_cell"
)

%%
listen := comms.Listen[int](toCell)

// Small javascript value that receives a number, increments and sends
// it back.
dom.TransientJavascript(
            fmt.Sprintf(`
(() => {
    let gonb_comm = globalThis?.gonb_comm;
    if (gonb_comm) {
        var subscrition_id;
        let id = gonb_comm.subscribe("%s", (address, value) => {
            if (value >= 3) {
                gonb_comm.unsubscribe(subscription_id);
            }
            console.log(address+"->"+value);
            value = value + 1;
            gonb_comm.send("%s", value);
        });
        subscription_id = id;
    }
})();
`, toFrontEnd, toCell))

// Send sequence of numbers to frond-end.
go func() {
    time.Sleep(1 * time.Second)
    for ii := 1; ii < 5; ii++ {
        time.Sleep(300 * time.Millisecond)
        fmt.Printf("sent %d\n", ii);
        comms.Send(toFrontEnd, ii)        
    }
}()

// Print out replies.
for counter := range listen.C {
    fmt.Printf("got %d\n", counter)
    if counter >= 4 {
        break
    }
}

// Makes sure no more counts come through.
go func() {
    comms.Send(toFrontEnd, 10)
    time.Sleep(500 * time.Millisecond)
    fmt.Printf("closed\n")
    listen.Close()
}()

if counter, ok := <-listen.C; ok {
    fmt.Fprintf(os.Stderr, "Unexpected counter %d received !?\n", counter)
}
fmt.Printf("done\n")

sent 1
got 2
sent 2
got 3
sent 3
got 4
sent 4
closed
done


### Test `AddressChan[T].LatestOnly`

We'll send values faster than we read them, and we want to check that we read always the latest -- and don't get blocked.

In [6]:
%%
// Set timeout for program.
go func() { 
    time.Sleep(2 * time.Second)
    fmt.Println("timedout")
    os.Exit(1)
}() 

listen := comms.Listen[int](toCell).LatestOnly()
for ii := 0; ii <= 3; ii ++ {
    // Send 3 values from the front-end.
    dom.TransientJavascript(fmt.Sprintf(`
(() => {
    let gonb_comm = globalThis?.gonb_comm;
    if (gonb_comm) {
        gonb_comm.send("%s", %d);
    }
})();
`, toCell, ii))
    time.Sleep(100 * time.Millisecond)
}

time.Sleep(500 * time.Millisecond)
latest := <-listen.C
if latest == 3 {
    fmt.Println("ok")
} else {
    fmt.Printf("error: received %d\n", latest)
}
listen.Close()

ok


#### Make Last Cell Quick To Execute

In [7]:
%%
fmt.Println("No more.")

No more.
